In [2]:
using LinearAlgebra
using Optim
using Plots
using SparseArrays
using IterativeSolvers
using DoubleFloats

# Set up the points
p1 = [0.0, 0.0]
p2 = [1.0, 0.0]
p3 = [0.0, 1.0]

m1 = (p2 .+ p3) ./ 2
m2 = (p1 .+ p3) ./ 2
m3 = (p1 .+ p2) ./ 2

b = (p1 .+ p2 .+ p3) ./ 3

# vertices
T1 = hcat(p1, p2, p3)
eval_T1(i, j) = T1[1, :] .^ i .* T1[2, :] .^ j

# edge midpoints
T2 = hcat(m1, m2, m3)
eval_T2(i, j) = T2[1, :] .^ i .* T2[2, :] .^ j

# midpoint
T3 = hcat(b)
eval_T3(i, j) = T3[1, :] .^ i .* T3[2, :] .^ j

# edge class
T4(a) = hcat(a * p1 .+ (1 .- a) * p2,
             a * p2 .+ (1 .- a) * p1,
             a * p3 .+ (1 .- a) * p1,
             a * p1 .+ (1 .- a) * p3,
             a * p3 .+ (1 .- a) * p2,
             a * p2 .+ (1 .- a) * p3)

eval_T4(a, i, j) = T4(a)[1, :] .^ (0 * (i < 0) + i * (i >= 0)) .* T4(a)[2, :] .^ (0 * (j < 0) + j * (j >= 0))

dT4 = hcat(p1 .- p2, p2 .- p1, p3 .- p1, p1 .- p3, p3 .- p2, p2 .- p3)
eval_dT4(i, j) = dT4[1, :] .^ (0 * (i < 0) + i * (i >= 0)) .* dT4[2, :] .^ (0 * (j < 0) + j * (j >= 0))

# inner class, type 1
T5(a) = hcat(a * m1 .+ (1 .- a) * p1, a * m2 .+ (1 .- a) * p2, a * m3 .+ (1 .- a) * p3)
dT5 = hcat(m1 .- p1, m2 .- p2, m3 .- p3)
eval_T5(a, i, j) = T5(a)[1, :] .^ (0 * (i < 0) + i * (i >= 0)) .* T5(a)[2, :] .^ (0 * (j < 0) + j * (j >= 0))

# inner class, type 2 (TODO)
T6(a, b) = hcat(b * (a * m1 .+ (1 .- a) * p1) .+ (1 .- b) * (a * m2 .+ (1 .- a) * p2),
                b * (a * m1 .+ (1 .- a) * p1) .+ (1 .- b) * (a * m3 .+ (1 .- a) * p3),
                b * (a * m3 .+ (1 .- a) * p3) .+ (1 .- b) * (a * m2 .+ (1 .- a) * p2),
                b * (a * m2 .+ (1 .- a) * p2) .+ (1 .- b) * (a * m1 .+ (1 .- a) * p1),
                b * (a * m3 .+ (1 .- a) * p3) .+ (1 .- b) * (a * m1 .+ (1 .- a) * p1),
                b * (a * m2 .+ (1 .- a) * p2) .+ (1 .- b) * (a * m3 .+ (1 .- a) * p3))

# Integral function
function integral(i, j)
    factorial(j) * factorial(i) / factorial(i + j + 2)
end

# System setup
function Aij(i, j, a)
    vcat(sum(eval_T1(i, j)),
         sum(eval_T4(a[1], i, j)),
         sum(eval_T5(a[2], i, j)),
         sum(eval_T5(a[3], i, j)))
end

function A(a)
    hcat(
        Aij(0,0,a),
                    
        Aij(1,0,a),
        # Aij(0,1,a),

        # Aij(1,1,a),
        Aij(2,0,a),
        # Aij(0,2,a),

        Aij(2,1,a),
        # Aij(1,2,a),
        Aij(3,0,a),
        # Aij(0,3,a),

        # Aij(2,2,a),
        Aij(1,3,a),
        # Aij(3,1,a),
        Aij(4,0,a),
        # Aij(0,4,a),

        Aij(1,4,a),
        # Aij(4,1,a),
        Aij(2,3,a),
        # Aij(3,2,a),
        Aij(0,5,a),
        # Aij(5,0,a),

        Aij(1,5,a),
        # Aij(5,1,a),
        Aij(2,4,a),
        # Aij(4,2,a),
        # Aij(3,3,a),
        Aij(0,6,a),
        # Aij(6,0,a),
         )
end

rhs = [
        integral(0,0),
                
        integral(1,0),
        # integral(0,1),

        # integral(1,1),
        integral(2,0),
        # integral(0,2),

        integral(2,1),
        # integral(1,2),
        integral(3,0),
        # integral(0,3),

        # integral(2,2),
        integral(1,3),
        # integral(3,1),
        integral(4,0),
        # integral(0,4),

        integral(1,4),
        # integral(4,1),
        integral(2,3),
        # integral(3,2),
        integral(0,5),
        # integral(5,0),

        integral(1,5),
        # integral(5,1),
        integral(2,4),
        # integral(4,2),
        # integral(3,3),
        integral(0,6),
        # integral(6,0),
       ];

In [2]:
f(a) = norm(A(a)'*((A(a) * A(a)')\(A(a) * rhs))-rhs)

f (generic function with 1 method)

In [3]:
f([0.3,0.2,0.1])

0.0014510545603040725

In [4]:
result = optimize(f, [0,0,0], [1,1,1], [0.1, 0.12, 0.13], Fminbox(NelderMead()),  Optim.Options(g_tol = 1e-15))

 * Status: success

 * Candidate solution
    Final objective value:     2.652415e-15

 * Found with
    Algorithm:     Fminbox with Nelder-Mead

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 8.79e-03 ≰ 1.0e-15

 * Work counters
    Seconds run:   3  (vs limit Inf)
    Iterations:    4
    f(x) calls:    1611
    ∇f(x) calls:   1


In [5]:
Optim.minimizer(result)

3-element Vector{Float64}:
 0.3077459416060209
 0.8506802519851394
 0.23722737279251319

In [1]:
using LinearAlgebra
using Optim
using Plots
using SparseArrays
using IterativeSolvers
using DoubleFloats

function factorial64(x::Double64)
    if x == 0
        return Double64(1.0)
    else
        return x * factorial64(x - 1)
    end
end

# Set up the points with Double64
p1 = Double64[0.0, 0.0]
p2 = Double64[1.0, 0.0]
p3 = Double64[0.0, 1.0]

m1 = (p2 .+ p3) ./ 2
m2 = (p1 .+ p3) ./ 2
m3 = (p1 .+ p2) ./ 2

b = (p1 .+ p2 .+ p3) ./ 3

# vertices
T1 = hcat(p1, p2, p3)
eval_T1(i, j) = T1[1, :] .^ Double64(i) .* T1[2, :] .^ Double64(j)

# edge midpoints
T2 = hcat(m1, m2, m3)
eval_T2(i, j) = T2[1, :] .^ Double64(i) .* T2[2, :] .^ Double64(j)

# midpoint
T3 = hcat(b)
eval_T3(i, j) = T3[1, :] .^ Double64(i) .* T3[2, :] .^ Double64(j)

# edge class
T4(a) = hcat(a * p1 .+ (1 .- a) * p2,
             a * p2 .+ (1 .- a) * p1,
             a * p3 .+ (1 .- a) * p1,
             a * p1 .+ (1 .- a) * p3,
             a * p3 .+ (1 .- a) * p2,
             a * p2 .+ (1 .- a) * p3)

eval_T4(a, i, j) = T4(a)[1, :] .^ (0 * (i < 0) + i * (i >= 0)) .* T4(a)[2, :] .^ (0 * (j < 0) + j * (j >= 0))

dT4 = hcat(p1 .- p2, p2 .- p1, p3 .- p1, p1 .- p3, p3 .- p2, p2 .- p3)
eval_dT4(i, j) = dT4[1, :] .^ (0 * (i < 0) + i * (i >= 0)) .* dT4[2, :] .^ (0 * (j < 0) + j * (j >= 0))

# inner class, type 1
T5(a) = hcat(a * m1 .+ (1 .- a) * p1, a * m2 .+ (1 .- a) * p2, a * m3 .+ (1 .- a) * p3)
dT5 = hcat(m1 .- p1, m2 .- p2, m3 .- p3)
eval_T5(a, i, j) = T5(a)[1, :] .^ (0 * (i < 0) + i * (i >= 0)) .* T5(a)[2, :] .^ (0 * (j < 0) + j * (j >= 0))

# Integral function
function integral(i, j)
    factorial64(Double64(j)) * factorial64(Double64(i)) / factorial64(Double64(i + j + 2))
end

# System setup
function Aij(i, j, a)
    vcat(sum(eval_T1(i, j)),
         sum(eval_T4(a[1], i, j)),
         sum(eval_T5(a[2], i, j)),
         sum(eval_T5(a[3], i, j)))
end

function A(a)
    hcat(Aij(0, 0, a),
         Aij(1, 0, a),
         Aij(2, 0, a),
         Aij(2, 1, a),
         Aij(3, 0, a),
         Aij(1, 3, a),
         Aij(4, 0, a),
         Aij(1, 4, a),
         Aij(2, 3, a),
         Aij(0, 5, a),
         Aij(1, 5, a),
         Aij(2, 4, a),
         Aij(0, 6, a))
end

rhs = [integral(0, 0),
       integral(1, 0),
       integral(2, 0),
       integral(2, 1),
       integral(3, 0),
       integral(1, 3),
       integral(4, 0),
       integral(1, 4),
       integral(2, 3),
       integral(0, 5),
       integral(1, 5),
       integral(2, 4),
       integral(0, 6)];

In [2]:
f(a) = norm(A(a)'*((A(a) * A(a)')\(A(a) * rhs))-rhs)

f (generic function with 1 method)

In [18]:
# Define the bounds
lower_bounds = Double64[0.0, 0.0, 0.0]
upper_bounds = Double64[Inf, Inf, Inf]

# f(a) = Double64[norm(A(a)'*((A(a) * A(a)')\(A(a) * rhs))-rhs)]

# Set up the optimization problem with bounds
result = optimize(f, lower_bounds, upper_bounds, Double64[0.1, 0.1, 0.2], Fminbox(NelderMead()), Optim.Options(g_tol = 1e-31, iterations = 100_000, show_trace = false))


In [14]:
x = Optim.minimizer(result)

3-element Vector{Double64}:
 6.9225405837400835389538372619186903e-01
 2.37227372793185736381326795590560855e-01
 8.5068025197949430405086231098419421e-01

In [1]:
using LinearAlgebra
using Optim
using Plots

# Custom factorial function for BigFloat
function factorial_big(x::BigFloat)
    if x == BigFloat(0)
        return BigFloat(1.0)
    else
        return x * factorial_big(x - BigFloat(1))
    end
end

# Set up the points with BigFloat
p1 = BigFloat[0.0, 0.0]
p2 = BigFloat[1.0, 0.0]
p3 = BigFloat[0.0, 1.0]

m1 = (p2 .+ p3) ./ BigFloat(2.0)
m2 = (p1 .+ p3) ./ BigFloat(2.0)
m3 = (p1 .+ p2) ./ BigFloat(2.0)

b = (p1 .+ p2 .+ p3) ./ BigFloat(3.0)

# vertices
T1 = hcat(p1, p2, p3)
eval_T1(i, j) = T1[1, :] .^ BigFloat(i) .* T1[2, :] .^ BigFloat(j)

# edge midpoints
T2 = hcat(m1, m2, m3)
eval_T2(i, j) = T2[1, :] .^ BigFloat(i) .* T2[2, :] .^ BigFloat(j)

# midpoint
T3 = hcat(b)
eval_T3(i, j) = T3[1, :] .^ BigFloat(i) .* T3[2, :] .^ BigFloat(j)

# edge class
T4(a) = hcat(a * p1 .+ (1 .- a) * p2,
             a * p2 .+ (1 .- a) * p1,
             a * p3 .+ (1 .- a) * p1,
             a * p1 .+ (1 .- a) * p3,
             a * p3 .+ (1 .- a) * p2,
             a * p2 .+ (1 .- a) * p3)

eval_T4(a, i, j) = T4(a)[1, :] .^ (0 * (i < 0) + BigFloat(i) * (i >= 0)) .* T4(a)[2, :] .^ (0 * (j < 0) + BigFloat(j) * (j >= 0))

dT4 = hcat(p1 .- p2, p2 .- p1, p3 .- p1, p1 .- p3, p3 .- p2, p2 .- p3)
eval_dT4(i, j) = dT4[1, :] .^ (0 * (i < 0) + BigFloat(i) * (i >= 0)) .* dT4[2, :] .^ (0 * (j < 0) + BigFloat(j) * (j >= 0))

# inner class, type 1
T5(a) = hcat(a * m1 .+ (1 .- a) * p1, a * m2 .+ (1 .- a) * p2, a * m3 .+ (1 .- a) * p3)
dT5 = hcat(m1 .- p1, m2 .- p2, m3 .- p3)
eval_T5(a, i, j) = T5(a)[1, :] .^ (0 * (i < 0) + BigFloat(i) * (i >= 0)) .* T5(a)[2, :] .^ (0 * (j < 0) + BigFloat(j) * (j >= 0))

# Integral function using custom factorial_big
function integral(i, j)
    factorial_big(BigFloat(j)) * factorial_big(BigFloat(i)) / factorial_big(BigFloat(i + j + 2))
end

# System setup
function Aij(i, j, a)
    vcat(sum(eval_T1(i, j)),
         sum(eval_T4(a[1], i, j)),
         sum(eval_T5(a[2], i, j)),
         sum(eval_T5(a[3], i, j)))
end

function A(a)
    hcat(Aij(0, 0, a),
         Aij(1, 0, a),
         Aij(2, 0, a),
         Aij(2, 1, a),
         Aij(3, 0, a),
         Aij(1, 3, a),
         Aij(4, 0, a),
         Aij(1, 4, a),
         Aij(2, 3, a),
         Aij(0, 5, a),
         Aij(1, 5, a),
         Aij(2, 4, a),
         Aij(0, 6, a))
end

rhs = [integral(0, 0),
       integral(1, 0),
       integral(2, 0),
       integral(2, 1),
       integral(3, 0),
       integral(1, 3),
       integral(4, 0),
       integral(1, 4),
       integral(2, 3),
       integral(0, 5),
       integral(1, 5),
       integral(2, 4),
       integral(0, 6)];

In [17]:
f(a) = norm(A(a)'*((A(a) * A(a)')\(A(a) * rhs))-rhs)

# a = (0.1,0.2,0.1)
# cond((A(a) * A(a)'), Inf)

f (generic function with 1 method)

In [3]:
# Define the bounds
lower_bounds = BigFloat[0.0, 0.0, 0.0]
upper_bounds = BigFloat[Inf, Inf, Inf]

# f(a) = Double64[norm(A(a)'*((A(a) * A(a)')\(A(a) * rhs))-rhs)]

# Set up the optimization problem with bounds
result = optimize(f, lower_bounds, upper_bounds, BigFloat[0.1, 0.1, 0.2], Fminbox(NelderMead()), Optim.Options(g_tol = 1e-63, iterations = 100_000, show_trace = false))


 * Status: success

 * Candidate solution
    Final objective value:     6.988051e-64

 * Found with
    Algorithm:     Fminbox with Nelder-Mead

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 9.29e-03 ≰ 1.0e-63

 * Work counters
    Seconds run:   33  (vs limit Inf)
    Iterations:    4
    f(x) calls:    7935
    ∇f(x) calls:   1


In [4]:
x = Optim.minimizer(result)

3-element Vector{BigFloat}:
 0.6922540583740083538953837157537490399617063915677563020138905937798600138656936
 0.2372273727931857363813267950086108322143854707884946004397634693057437204509927
 0.8506802519794943040508623079882848864972963422938270486893607218359283319043889

In [5]:
1-x[1]

0.3077459416259916461046162842462509600382936084322436979861094062201399861343064